<a href="https://colab.research.google.com/github/Denev6/practice/blob/main/transformer/RoBERTa_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

import nltk

nltk.download("punkt")

In [ ]:
import os
import re
import gc
import warnings

import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizerFast, RobertaModel
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount("/content/drive")
warnings.filterwarnings("ignore")

In [ ]:
!python --version

print(f"torch: {torch.__version__}")

Python 3.8.15
torch: 1.12.1+cu113


In [ ]:
def join_path(*args):
    return os.path.join("/content/drive/MyDrive/DACON", *args)


DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
TRAIN_CSV = join_path("data", "train.csv")
TEST_CSV = join_path("data", "test.csv")

ARGS = {
    "model": "tae898/emoberta-large",
    "model_path": join_path("emoberta", "model.pth"),
    "batch_size": 64,
    "epochs": 100,
    "max_len": 256,
    "lr": 2e-4,
    "min_lr": 1e-6,
    "stopping_patience": 5,
    "scheduler_patience": 2,
}

In [ ]:
class EarlyStoppingCallback(object):
    def __init__(self, patience=2):
        self._min_eval_loss = np.inf
        self._patience = patience
        self.__counter = 0

    def should_stop(self, eval_loss, model, save_path):
        if eval_loss < self._min_eval_loss:
            self._min_eval_loss = eval_loss
            self.__counter = 0
            torch.save(model.state_dict(), save_path)
        elif eval_loss > self._min_eval_loss:
            self.__counter += 1
            if self.__counter >= self._patience:
                return True
        return False

In [ ]:
class EmoLabelEncoder(object):
    def __init__(self):
        self._targets = [
            "neutral",
            "joy",
            "surprise",
            "anger",
            "sadness",
            "disgust",
            "fear",
        ]
        self.num_classes = len(self._targets)

    def encode(self, label):
        return self._targets.index(label)

    def decode(self, label):
        return self._targets[label]

In [ ]:
class EmoDataset(Dataset):
    def __init__(
        self,
        data,
        twt_tokenizer,
        roberta_tokenizer,
        label_encoder,
        max_length=256,
        mode=None,
    ):
        self._label_encoder = label_encoder
        self._twt_tokenizer = twt_tokenizer
        self._roberta_tokenizer = roberta_tokenizer
        self._max_length = max_length
        self._mode = mode
        self._dataset = self._init_dataset(data)

    def _init_dataset(self, data):
        if self._mode == "train":
            data["Target"] = data["Target"].map(self._label_encoder.encode)
            data = data.loc[:, ["Utterance", "Target"]]
        else:
            data = data.loc[:, "Utterance"]
            data = data.to_frame()
        return data

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, idx):
        text = self._dataset.loc[idx, "Utterance"]
        inputs = self._roberta_tokenizer(
            text,
            max_length=self._max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        input_ids = inputs["input_ids"][0]
        attention_mask = inputs["attention_mask"][0]

        if self._mode == "train":
            y = self._dataset.loc[idx, "Target"]
            return input_ids, attention_mask, y
        else:
            return input_ids, attention_mask

In [ ]:
class EmoClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EmoClassifier, self).__init__()
        self.hidden_size = 128
        self.emoberta = RobertaModel.from_pretrained(ARGS["model"], add_pooling_layer=False)
        self.gru = nn.GRU(input_size=1024, hidden_size=self.hidden_size, dropout=0.1, batch_first=True)
        self.fc = nn.Linear(self.hidden_size, num_classes)
        self.out = nn.Softmax(dim=-1)

        for _, param in self.emoberta.named_parameters():
            param.requires_grad = False

    def forward(self, id, mask):
        output = self.emoberta(id, mask)
        output, _ = self.gru(output.last_hidden_state)
        output = self.fc(output[:, -1, :])
        output = self.out(output)

        return output

# Dataset

In [ ]:
train_csv = pd.read_csv(TRAIN_CSV)

df_train, df_val = train_test_split(
    train_csv, test_size=0.2, shuffle=False
)
df_train.head()

In [ ]:
label_encoder = EmoLabelEncoder()
roberta_tokenizer = RobertaTokenizerFast.from_pretrained(ARGS["model"], truncation=True)
twt_tokenizer = nltk.tokenize.TweetTokenizer(
    preserve_case=False, strip_handles=True, reduce_len=True
)

train_set = EmoDataset(
    df_train.reset_index(drop=True),
    twt_tokenizer,
    roberta_tokenizer,
    label_encoder,
    max_length=ARGS["max_len"],
    mode="train",
)
val_set = EmoDataset(
    df_val.reset_index(drop=True),
    twt_tokenizer,
    roberta_tokenizer,
    label_encoder,
    max_length=ARGS["max_len"],
    mode="train",
)

train_dataloader = DataLoader(train_set, batch_size=ARGS["batch_size"])
val_dataloader = DataLoader(val_set, batch_size=ARGS["batch_size"])

# Model

In [ ]:
def evaluate(model, criterion, val_loader, device, mode=None):
    model.eval()

    val_loss = list()
    model_preds = list()
    true_labels = list()

    with torch.no_grad():
        for input_ids, attention_mask, label in val_loader:
            label = label.to(device)
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output, label.long())
            val_loss.append(batch_loss.item())

            if mode != "train":
                model_preds += output.argmax(1).detach().cpu().numpy().tolist()
                true_labels += label.detach().cpu().numpy().tolist()

        if mode != "train":
            val_acc = accuracy_score(true_labels, model_preds)
            val_f1 = f1_score(true_labels, model_preds, average="macro")
            return val_acc, val_f1

        return val_loss


def train(model, optimizer, scheduler, criterion, train_loader, val_loader, device):
    torch.cuda.empty_cache()
    gc.collect()

    epoch_progress = trange(1, ARGS["epochs"] + 1)
    early_stopper = EarlyStoppingCallback(patience=ARGS["stopping_patience"])
    criterion.to(device)

    model_path = ARGS["model_path"]
    model.to(device)
    model.zero_grad()

    for epoch in epoch_progress:

        model.train()
        train_loss = list()
        for data in train_loader:

            input_ids, attention_mask, train_label = data
            train_label = train_label.to(device)
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output, train_label.long())
            train_loss.append(batch_loss.item())

            batch_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        val_loss = evaluate(model, criterion, val_loader, device, mode="train")
        train_loss = np.mean(train_loss)
        val_loss = np.mean(val_loss)
        tqdm.write(
            f"Epoch {epoch},  Train-Loss: {train_loss:.5f},  Val-Loss: {val_loss:.5f}"
        )

        if early_stopper.should_stop(val_loss, model, model_path):
            tqdm.write(f"\n\n -- EarlyStoppingCallback: [Epoch: {epoch - ARGS['stopping_patience']}]")
            tqdm.write(f"Model saved at '{model_path}'.")
            break

        scheduler.step(val_loss)
        
    model.load_state_dict(torch.load(model_path))
    return model

In [ ]:
model = EmoClassifier(label_encoder.num_classes)
optimizer = optim.NAdam(
    model.parameters(), lr=ARGS["lr"], weight_decay=0.01
)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.7, patience=ARGS["scheduler_patience"], min_lr =["min_lr"])
criterion = nn.CrossEntropyLoss()

Some weights of the model checkpoint at tae898/emoberta-large were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
best_model = train(
    model, optimizer, scheduler, criterion, train_dataloader, val_dataloader, DEVICE
)

In [ ]:
torch.cuda.empty_cache()
gc.collect()

val_acc, val_f1 = evaluate(model, criterion, val_dataloader, DEVICE)
print(f"Accuracy: {val_acc:.5f}")
print(f"F1-macro: {val_f1:.5f}")

# Prediction

In [ ]:
df_test = pd.read_csv(TEST_CSV)
test_set = EmoDataset(
    df_test.reset_index(drop=True),
    twt_tokenizer,
    roberta_tokenizer,
    label_encoder,
    max_length=ARGS["max_len"],
)
test_dataloader = DataLoader(test_set, batch_size=ARGS["batch_size"], shuffle=False)

In [ ]:
def predict(model, test_loader, device):
    model.eval()
    model_preds = list()

    with torch.no_grad():
        for input_ids, attention_mask in test_loader:
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)

            output = model(input_id, mask)
            model_preds += output.argmax(1).detach().cpu().numpy().tolist()
        return model_preds

In [ ]:
preds = predict(model, test_dataloader, DEVICE)

In [ ]:
df_test["Target"] = preds
df_test["Target"] = df_test["Target"].map(label_encoder.decode)
result = df_test.loc[:, ["ID", "Target"]]
result.head()